In [1]:
from transformers import T5ForConditionalGeneration, AutoTokenizer, BitsAndBytesConfig
import torch

repo_name = "fristrup/flan-t5-semantic-tagger-base-4bit"


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(repo_name)
model = T5ForConditionalGeneration.from_pretrained(
    repo_name, quantization_config=quantization_config, device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


config.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

/mnt/d/repos/github/QuoteWeave/models/.venv/lib/python3.11/site-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/333M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [4]:
import pandas as pd

In [2]:
def predict_tags(model, tokenizer, text, max_length=128):
    """
    Generate tags for a given text using the trained model
    """
    # Prepare the input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    input_ids = inputs.input_ids

    # Move to the same device as model
    device = model.device
    input_ids = input_ids.to(device)

    # Generate tags
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            early_stopping=True,
        )

    # Decode the generated tokens
    predicted_tags = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean up tags - split by comma and strip whitespace
    tags = [tag.strip() for tag in predicted_tags.split(",") if tag.strip()]

    # Remove duplicates while preserving order
    unique_tags = []
    for tag in tags:
        if tag not in unique_tags:
            unique_tags.append(tag)

    return unique_tags

In [5]:
df = pd.read_csv("hf://datasets/jstet/quotes-500k/quotes.csv")

In [8]:
predicted_tags = predict_tags(model, tokenizer, df["quote"].iloc[0])

In [9]:
print(predicted_tags)

["I'm selfish", 'impatient and a little insecure']
